# Simulation notebook - Colombia data

This notebook generates a people dataframe with 500k+ people from the manzanas data, and runs the HMM-based simualtion with fixed transitions over them.

In [173]:
import sys
import os
sys.path.append("../..")
import model 
import geopandas as gpd
import pandas as pd
import numpy as np
import scipy
from model.sim import location, state, contacts
from IPython.display import display
from timeit import default_timer as timer
import swifter
from tqdm import tqdm_notebook as tqdm
import dask.dataframe as dd
from dask.multiprocessing import get

In [113]:
crs = {'epsg:3857'}
shp_path = '../../../shp' # Change to your local shape file directory
# os.listdir(shp_path)

In [124]:
mnz_data = gpd.read_file(os.path.join(shp_path,"Censo_personas_manzanas_2018.shp"))
list(mnz_data.columns)

['DEPTO',
 'MPIO',
 'U_DPTO',
 'U_MPIO',
 'UA_CLASE',
 'UA2_CPOB',
 'UASECT_URB',
 'UASECC_URB',
 'UA_MANZANA',
 'ID_UNIFICA',
 'U_AG',
 'SEXO_M',
 'SEXO_H',
 'SEXO_TOTAL',
 'EDAD_0_4',
 'EDAD_5_9',
 'EDAD_10_14',
 'EDAD_15_19',
 'EDAD_20_24',
 'EDAD_25_29',
 'EDAD_30_34',
 'EDAD_35_39',
 'EDAD_40_44',
 'EDAD_45_49',
 'EDAD_50_54',
 'EDAD_55_59',
 'EDAD_60_64',
 'EDAD_65_69',
 'EDAD_70_74',
 'EDAD_75_79',
 'EDAD_80_84',
 'EDAD_85_89',
 'EDAD_90_94',
 'EDAD_95_99',
 'EDAD_100_O',
 'TOTAL_MAYO',
 'GRUPO_ETNI',
 'GRUPO_ET_1',
 'GRUPO_ET_2',
 'GRUPO_ET_3',
 'GRUPO_ET_4',
 'GRUPO_ET_5',
 'GRUPO_ET_6',
 'LUG_NAC_ES',
 'LUG_NAC_OT',
 'LUG_NAC__1',
 'LUG_NAC_NO',
 'TRATAMIENT',
 'TRATAMIE_1',
 'TRATAMIE_2',
 'TRATAMIE_3',
 'TRATAMIE_4',
 'TRATAMIE_5',
 'TRATAMIE_6',
 'TRATAMIE_7',
 'TRATAMIE_8',
 'TRATAMIE_9',
 'LO_ATENDIE',
 'LO_ATEND_1',
 'LO_ATEND_2',
 'CALIDAD_SE',
 'CALIDAD__1',
 'CALIDAD__2',
 'CALIDAD__3',
 'CONDICION_',
 'CONDICION1',
 'ALFABETA_S',
 'ALFABETA_N',
 'ALFABETA_1',
 'ASIS

In [117]:
print("Number of manzanas:", len(manzanas))
print("Number of people separated in manzanas:", manzanas['SEXO_TOTAL'].sum())

Number of manzanas: 5964
Number of people separated in manzanas: 534786


In [118]:
# utils functions  

def calculate_contacts(polygon, points, date):
    _people = points[points.geometry.centroid.within(polygon.geometry.buffer(100))]
    data = {
        'date': np.repeat(date, len(_people)),
        'patient1': np.repeat(polygon['patient'], len(_people)),
        'patient2': list(_people['patient'])
    }
    
    exposure_df = pd.DataFrame(data=data, columns = ["date","patient1","patient2"])
    return exposure_df

def extrapolation(df):
    transformation_table = []
    for i, value in df.sort_values("patient").iterrows():
        original_index = value['patient']
        new_index = 0
        step = value['SEXO_TOTAL']
        transformation_table[original_index] = [np.arange(new_index,new_index+step-1)]
        new_index = new_index + step
    return transformation_table

In [191]:
def instantiate_sim(neigh_info_path, people_df, T=100, reindex = True): 
    print("type(people_df), len(people_df)",type(people_df), len(people_df))
    sim = {"map":gpd.read_file(neigh_info_path)}
    pop_per_neigh = sim["map"]['SEXO_TOTAL']
    N0 = len(people_df)
    
    if reindex:
        people_df['patient'] = np.arange(len(people_df))
    #N0 = people.SEXO_TOTAL.sum()
    #people = people.reindex(people.index.repeat(people.SEXO_TOTAL))
             
    lat = people_df.position.apply(lambda point: point.y)
    lng = people_df.position.apply(lambda point: point.x)

#     people_df['calculated_centroid'] = people_df.centroid

    sim["location"] = pd.DataFrame(
        {
        "patient": np.repeat(np.arange(N0), T),
        "date": np.tile(np.arange(T), N0),
        "latitude": np.repeat(lat, T),
        "longitude": np.repeat(lng, T),
        }
    )

#     people_points = people_df.centroid
    distance_cutoff = 0.012
    sim["patients"] = pd.DataFrame({"patient": np.unique(sim["location"]["patient"])})
    sim["dates"] = pd.DataFrame({"date": np.unique(sim["location"]["date"])})
    
    return sim

# manz_info_path = shp_path + "/Censo_personas_manzanas_2018.shp"  
# manz = gpd.read_file(manz_info_path, index=None) 
# manz_proj = people.to_crs({"init": "EPSG:3857"})
# sim = instantiate_sim(manz_info_path, manz_proj)

In [187]:
large_people_df.position.apply(lambda point: point.x)

0        -8.322742e+06
1        -8.322742e+06
2        -8.322742e+06
3        -8.322742e+06
4        -8.322742e+06
              ...     
534781   -8.325669e+06
534782   -8.325669e+06
534783   -8.325669e+06
534784   -8.325669e+06
534785   -8.325669e+06
Name: position, Length: 534786, dtype: float64

In [198]:
def calculate_daily_contacts(people_proj, sim, load_from_csv=True):
    
    if load_from_csv:
        return pd.read_csv("improved_daily_contacts.csv")
    
    start = timer()

    # calculate daily contacts for day 1 as they will be repeated for the other days
    date = sim["dates"]["date"][0]
    daily_contacts = people_proj.apply(calculate_contacts, axis=1, args=[people_proj, date])

    original_daily_contacts = pd.concat(list(daily_contacts), sort=False)
    original_daily_contacts.reset_index(drop=True, inplace=True)

    # calculate the table for all the dates
    improved_daily_contacts = original_daily_contacts.copy()
    for date in sim["dates"]["date"]:
        if len(original_daily_contacts[original_daily_contacts['date'] == date]) < 1:
            new_df = original_daily_contacts.copy()
            new_df['date'] = date
            improved_daily_contacts = improved_daily_contacts.append(new_df)

    improved_daily_contacts.reset_index(drop=True, inplace=True)

    end = timer()
    print("Compute time:",end-start)
    
    return improved_daily_contacts

improved_daily_contacts = calculate_daily_contacts(people_proj, sim)
sim["contacts"] = improved_daily_contacts
sim["N_c"] = contacts.calculate_Nc(sim)
print("Average daily contacts: {}".format(sim["N_c"]))

KeyboardInterrupt: 

In [131]:
print(improved_daily_contacts["date"].max())

999


In [ ]:
save=False
if save:   
    improved_daily_contacts.to_csv("improved_daily_contacts.csv")

In [14]:
# Run sim
def run_sim(sim, N_infected=15, infected=None):
    sim["states"], sim["tests"] = state.simulate_states(sim, N_infected=N_infected, infected=infected)
    sim["hospital"] = state.get_first_occurrence(sim["states"], 6)
    sim["deaths"] = state.get_first_occurrence(sim["states"], 8)
    return sim
    
# run_sim(sim)

In [15]:
# gdf = gpd.GeoDataFrame(
#     df,
#     geometry=gpd.points_from_xy(
#         df["longitude"],
#         df["latitude"],
#     ),
#     crs={"init":"EPSG:4326"},
# )

# # 10 records
# filtered_df

# filtered_gdf = gpd.GeoDataFrame(
#     filtered_df, 
#     geometry=gpd.points_from_xy(
#         filtered_df["longitude"],
#         filtered_df["latitude"],
#     ),
#     crs={"init":"EPSG:4326"},
# )

# # EPSG:3857 converts it to meters, correct?

# gdf_proj = gdf.to_crs({"init": "EPSG:3857"})
# filtered_gdf_proj = filtered_gdf.to_crs({"init": "EPSG:3857"})

# # so 100 miles would be 160934 meters

# x = filtered_gdf_proj.buffer(100).unary_union

# neighbours = gdf_proj["geometry"].intersection(x)

# Simulation with 500k+ people 

In [144]:
# Build people_full dataframe containing census data, infected data + estimated people from neighborhood data

from tqdm import tqdm_notebook as tqdm

def create_people_df_from_mnz_data(mnz_data, create_contacts=True):
#     people_df = dummy_row
#     people_df.patient = [0]

    people_df = pd.DataFrame()
    contacts_df_list = []
    i = 0
    first_pass = True
    for idx, row in tqdm(mnz_data.iterrows()):
        n_ppl_in_mnz = row["SEXO_TOTAL"]
        if not n_ppl_in_mnz:
            continue
        
#         print("row.geometry.centroid",row.geometry.centroid)
#         dummy_row.geometry = row.geometry
        person_row = pd.DataFrame({"patient": [0], "position": [row.geometry.centroid]})
        
#         dummy_row["calculated_centroid"] = row.geometry.centroid
#         print(dummy_row.columns)
        ppl_in_mnz = pd.concat([person_row]*n_ppl_in_mnz)
        ids = np.arange(i, i+n_ppl_in_mnz)
        ppl_in_mnz.patient = ids
        i+=n_ppl_in_mnz
        if first_pass:
            people_df = ppl_in_mnz
            first_pass = False
        else:
            people_df = people_df.append(ppl_in_mnz, ignore_index=True)
            
        
        if create_contacts:
            for date in range(90):
                for id_ in ids:
                        data = {
                            'date': np.repeat(date, n_ppl_in_mnz),
                            'patient1': np.repeat(id_, n_ppl_in_mnz),
                            'patient2': ids
                        }
            contacts_df_list.append(pd.DataFrame(data=data, columns = ["date", "patient1", "patient2"]))
    
    contacts_df = pd.concat(contacts_df_list, sort=False)
    contacts_df.reset_index(drop=True, inplace=True)
    
    print("Final shape of people_df:",people_df.shape)
    return people_df, contacts_df

sim["map"]["calculated_centroid"] = sim["map"].centroid
large_people_df, contacts_df = create_people_df_from_mnz_data(mnz_data)

/home/camilo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]



Final shape of people_df: (534786, 2)


In [169]:
large_people_df.to_csv("people_df_from_manzanas.csv")

In [154]:
def get_mnz2ids(mnz_data):
    i = 0
    mnz2ids = {}
    for idx, row in tqdm(mnz_data.iterrows()):
        n_ppl_in_mnz = row["SEXO_TOTAL"]
        mnz2ids[idx] =  np.arange(i, i+n_ppl_in_mnz)
        i += n_ppl_in_mnz
        
    return mnz2ids

mnz2ids = get_mnz2ids(mnz_data)

/home/camilo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [151]:
print("Shape of contacts df (should be people_df*90*avg_n_contacts)", contacts_df.shape)
contacts_df.iloc[3]

Shape of contacts df (should be people_df*90*avg_n_contacts) (534786, 3)


date        89
patient1    15
patient2     3
Name: 3, dtype: int64

# Extrapolate contacts

In [145]:
from shapely.ops import nearest_points

# def nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
#     """Find the nearest point and return the corresponding value from specified column."""
#     # Find the geometry that is closest
#     nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
#     # Get the corresponding value from df2 (matching is based on the geometry)
#     value = df2[nearest][src_column].get_values()[0]
#     return value

def nearest(point, people_df):
    people_locs = people_df.geometry.centroid.unary_union
    
#     print("Trying to find nearest point to %s" % point)
#     print("Candidates are:", people_locs[:5], "and more (%d total)" % len(people_locs)) # "with ids:", people_df["patient"][:5]
    
    # find the nearest person for which we have data
#     print("people_df.geometry.centroid", people_df.geometry.centroid)
#     print("nearest_points(point, people_locs)", nearest_points(point, people_locs))
    nearest = people_df.geometry.centroid == nearest_points(point, people_locs)[1]
#     print(nearest)
    
    return people_df[nearest]

def calculate_contacts(polygon, points, date):
    _people = points[points.geometry.centroid.within(polygon.geometry.buffer(100))]
    data = {
        'date': np.repeat(date, len(_people)),
        'patient1': np.repeat(polygon['patient'], len(_people)),
        'patient2': list(_people['patient'])
    }
    
    exposure_df = pd.DataFrame(data=data, columns = ["date", "patient1", "patient2"])
    return exposure_df

def contacts_from_closest_person(row, people, contacts):
    print("\nGetting contacts for person", row["patient"])
    nearest_person = nearest(row.calculated_centroid, people)
    contact_df = contacts[contacts['patient1']==nearest_person["patient"].iloc[0]]
    contact_df.patient1 = row["patient"]
#     print(contact_df)
    print("Found nearest person. ID: %s. Centroid: %s. Barrio: %s. Contacts: %d" % (nearest_person["patient"], nearest_person.geometry.centroid,
                                                                                   nearest_person.BARRIO, len(contact_df))) 
    
    return contact_df

def contacts_from_neigh(row, people, contacts):
    
    contacts_for_this_id = neigh2contacts[row[""]]
    return contacts

def dummy_contacts(row, people, contacts):
    contact_df = contacts.iloc[:30]
    contact_df.patient1 = row["patient"]
    data = {
        'date': contact_df["date"].to_numpy(),
        'patient1': contact_df["patient1"].to_numpy(),
        'patient2':  contact_df["patient2"].to_numpy()
    }
    contact_df = pd.DataFrame(data=data, columns = ["date", "patient1", "patient2"])
    return contact_df
    
def extrapolate_contacts(contacts, people, full_people):
    print("Extrapolating contacts for full dataframe of length %d from small dataframe of length %d" % (len(full_people), len(people)))
#     extrapolated_contacts = full_people.apply(contacts_from_closest_person, axis=1, args=[people, contacts])
    ddata = dd.from_pandas(full_people, npartitions=100)
    extrapolated_contacts = ddata.map_partitions(lambda df: df.apply(calculate_contacts, axis=1, args=[people, date])).compute(get=get) 
    return extrapolated_contacts
    

In [73]:
# print(improved_daily_contacts.columns)
# extrapolated_contacts = extrapolate_contacts(improved_daily_contacts, people_proj, large_people_df)

Index(['Unnamed: 0', 'date', 'patient1', 'patient2'], dtype='object')


In [21]:
# print("shape extrapolated contacts:", extrapolated_contacts.shape)

shape extrapolated contacts: (551560,)


In [171]:
# save=True
# if save:
#     extrapolated_contacts.to_csv("extrapolated_contacts.csv")
    
# load=False
# if load:
#     extrapolated_contacts = pd.read_csv("extrapolated_contacts.csv")

# Get list of infected

In [174]:

def include_infected(sim, path_to_infected_file=os.path.join(shp_path,"POSITIVOS_COVID_19.shp"), T=90, start_id=0):
    posi = gpd.read_file(path_to_infected_file)

    inf_active = posi[posi["estado_ate"]==1]
    latitudes = inf_active.geometry.centroid.y
    longitudes = inf_active.geometry.centroid.x
    infected_ids = np.arange(start_id, len(inf_active)+start_id)
    infected_locations = pd.DataFrame({"patient": np.tile(infected_ids, T), 
                                       "date": np.tile(np.arange(T), len(inf_active)), 
                                       "latitude": np.repeat(latitudes, T), 
                                       "longitude": np.repeat(longitudes, T)})
    
    sim["location"].append(infected_locations)
    sim["patients"] = sim["patients"].append(pd.DataFrame({"patient": infected_ids}))


    contacts_df_list = []
    c=0
    for i, mnz in tqdm(mnz_data.iterrows()):
#         print("searching if any infected in mnz %d" % i)
        infected_ids_in_mnz = infected_ids[inf_active.geometry.centroid.within(mnz.geometry.buffer(100))]
#         print("found %d infected in mnz" % len(infected_ids_in_mnz))
#         print(infected_ids_in_mnz)
        if len(infected_ids_in_mnz)==0:
            continue
        ids_in_mnz = mnz2ids[i]
        n_ppl_in_mnz = len(ids_in_mnz)
#         print("People in this mnz:", n_ppl_in_mnz)
        
        for date in range(90):
            for id_inf in infected_ids_in_mnz:
                data = {
                    'date': np.repeat(date, n_ppl_in_mnz),
                    'patient1': np.repeat(id_inf, n_ppl_in_mnz),
                    'patient2': ids_in_mnz
                }

                contacts_df_list.append(pd.DataFrame(data=data, columns = ["date", "patient1", "patient2"]))
                
        c+=1
        if c>100:break
    print("Number of contact dataframes (each correspond to the contacts of one infected on one date): %d" % len(contacts_df_list))
    infected_contacts = pd.concat(contacts_df_list)
    sim["contacts"].append(infected_contacts)
    return sim, infected_ids
    
# test
include_infected(sim_full)

/home/camilo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Number of contact dataframes (each correspond to the contacts of one infected on one date): 20970


({'map':       Id             BARRIO           area        COMUNA  \
  0      0       Las Gaviotas  103710.737439  Centro Norte   
  1      1       Los Balkanes   58750.503679  Centro Norte   
  2      2    Las Gaviotas II   80384.347238  Centro Norte   
  3      3   Villa Del Carmen  133642.372386  Centro Norte   
  4      4      Villa Cecilia   73397.793457  Centro Norte   
  ..   ...                ...            ...           ...   
  227  232  Urb. Doña Manuela   86397.705219           Sur   
  228  233     Matha  Gisella   17258.161905           Sur   
  229  234        Urb. El Rio  113650.306502      Oriental   
  230  199   Rigoberta Menchu   11870.886466      Oriental   
  231  212     Linda Maria II   12053.239505           Sur   
  
                                     GlobalID  CreationDa           Creator  \
  0    {DA37628A-E472-4563-A5D9-BAFBCE2FF0C5}  2020-05-11  alcaldia_soledad   
  1    {DA43EBEF-6C64-42D0-9C20-DD7BFC415CFD}  2020-05-11  alcaldia_soledad   
  2    {5

# Run full simulation

In [199]:
T=90
sim_full = instantiate_sim(neigh_info_path, large_people_df, T=T, reindex=True)
sim_full["contacts"] = contacts_df
print("Simulation instantiated. Current people in simulation: %d" % len(sim_full["patients"]))
print("Current simulation keys:", sim_full.keys())
sim_full, infected_ids = include_infected(sim_full, T=T, start_id=len(large_people_df))
print("Included %d infected" % len(infected_ids))
print("Final number of people to simulate: %d" % len(sim_full["patients"]))
sim_full["N_c"] = contacts.calculate_Nc(sim_full)
print("Average daily contacts for full df: {}".format(sim_full["N_c"]))
# np.savetxt('sim_states_full.csv', sim_results["states"], delimiter=',')

type(people_df), len(people_df) <class 'pandas.core.frame.DataFrame'> 534786
Simulation instantiated. Current people in simulation: 534786
Current simulation keys: dict_keys(['map', 'location', 'patients', 'dates', 'contacts'])


/home/camilo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Number of contact dataframes (each correspond to the contacts of one infected on one date): 20970
Included 456 infected
Final number of people to simulate: 535242
Average daily contacts for full df: 0.022203289975998394


In [197]:
sim_results = run_sim(sim_full, N_infected=0, infected = infected_ids)

Calculating states...
t = 0; 456 infectious; 0 exposed; 534786 susceptible; 0 dead
t = 9; 131 infectious; 0 exposed; 534786 susceptible; 0 dead
t = 18; 33 infectious; 0 exposed; 534786 susceptible; 0 dead
t = 27; 8 infectious; 0 exposed; 534786 susceptible; 0 dead
Calculated states in 54.31 seconds.


KeyboardInterrupt: 

In [63]:
np.savetxt('sim_states_full.csv', sim_results["states"], delimiter=',')

# Alternative: compute full contacts with heavy parallelism

In [189]:
def calculate_contacts(polygon, points, date):
    
    _people = points[points.geometry.centroid.within(polygon.geometry.buffer(100))]
#     print(len(_people))
    data = {
        'date': np.repeat(date, len(_people)),
        'patient1': np.repeat(polygon['patient'], len(_people)),
        'patient2': list(_people['patient'])
    }
    
    exposure_df = pd.DataFrame(data=data, columns = ["date", "patient1", "patient2"])
    return exposure_df

def calculate_daily_contacts(people_proj, sim, load_from_csv=False):
    
    if load_from_csv:
        return pd.read_csv("improved_daily_contacts.csv")
    
    start = timer()

    # calculate daily contacts for day 1 as they will be repeated for the other days
    date = sim["dates"]["date"][0]
    daily_contacts = people_proj.swifter.apply(calculate_contacts, axis=1, args=[people_proj, date])

    original_daily_contacts = pd.concat(list(daily_contacts), sort=False)
    original_daily_contacts.reset_index(drop=True, inplace=True)

    # calculate the table for all the dates
    improved_daily_contacts = original_daily_contacts.copy()
    for date in sim["dates"]["date"]:
        if len(original_daily_contacts[original_daily_contacts['date'] == date]) < 1:
            new_df = original_daily_contacts.copy()
            new_df['date'] = date
            improved_daily_contacts = improved_daily_contacts.append(new_df)

    improved_daily_contacts.reset_index(drop=True, inplace=True)

    end = timer()
    print("Compute time:",end-start)
    
    return improved_daily_contacts

full_daily_contacts = calculate_daily_contacts(large_people_df, sim_full, load_from_csv=False)
sim_full["contacts"] = full_daily_contacts
sim_full["N_c"] = contacts.calculate_Nc(sim_full)
print("Average daily contacts: {}".format(sim_full["N_c"]))

AttributeError: 'DataFrame' object has no attribute 'geometry'

In [86]:
print(full_daily_contacts)

          Unnamed: 0  date  patient1  patient2
0                  0     0         0         0
1                  1     0         0        15
2                  2     0         0      1479
3                  3     0         0      1480
4                  4     0         1         1
...              ...   ...       ...       ...
98415995    98415995   999      5963      5959
98415996    98415996   999      5963      5960
98415997    98415997   999      5963      5961
98415998    98415998   999      5963      5962
98415999    98415999   999      5963      5963

[98416000 rows x 4 columns]


In [90]:
sim_results = run_sim(sim_full, N_infected=0, infected = infected_ids)

Calculating states...
t = 0; 456 infectious; 0 exposed; 551560 susceptible; 0 dead
t = 9; 134 infectious; 0 exposed; 551560 susceptible; 1 dead
t = 18; 30 infectious; 0 exposed; 551560 susceptible; 2 dead
t = 27; 7 infectious; 0 exposed; 551560 susceptible; 2 dead
t = 36; 1 infectious; 0 exposed; 551560 susceptible; 3 dead
t = 45; 0 infectious; 0 exposed; 551560 susceptible; 4 dead
t = 54; 0 infectious; 0 exposed; 551560 susceptible; 4 dead
t = 63; 0 infectious; 0 exposed; 551560 susceptible; 4 dead
t = 72; 0 infectious; 0 exposed; 551560 susceptible; 4 dead
t = 81; 0 infectious; 0 exposed; 551560 susceptible; 4 dead
Calculated states in 505.66 seconds.
